In [1]:
! pip install python-docx
! pip install urllib2
! pip install Pillow

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Could not find a version that satisfies the requirement urllib2 (from versions: )
No matching distribution found for urllib2
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from docx import *
from datetime import datetime, timedelta
from docx import Document
from docx.shared import Inches
import zipfile
import datetime
from datetime import *
import json
from urllib import request
from urllib.request import urlopen
import pandas as pd
from io import StringIO
import urllib
import csv
from pprint import pprint
import subprocess

def get_week(date=datetime.now()):
    #date = datetime.strptime("2018-08-31","%Y-%m-%d")
    one_day = timedelta(days=1)
    day_idx = (date.weekday() + 1) % 7  
    monday = date - timedelta(days=day_idx-1)
    date = monday
    for n in range(0,5):
        yield date.strftime('%d-%b-%Y')
        date += one_day

def state_holiday(str_val):
    aus_state = ['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA']
    my_holiday_lst = str_val.strip().split("|")
    if "NAT" in my_holiday_lst:
        my_holiday_lst = aus_state
    my_holiday_lst.sort()    
    da = [1 if i in my_holiday_lst else 0 for i in aus_state]    
    return ",".join(str(ele) for ele in da)

def get_data_api(in_date):
    url = 'https://data.gov.au/dataset/b1bc6077-dadd-4f61-9f8c-002ab2cdff10/resource/253d63c0-af1f-4f4c-b8d5-eb9d9b1d46ab/download/australianpublicholidays-201718.csv'
    aus_holiday = pd.read_csv(url)
    aus_holiday[['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA']] = aus_holiday["Applicable To"].apply(state_holiday).apply(lambda x: pd.Series(str(x).split(',')))
    return len(aus_holiday[(aus_holiday.Date == in_date) & (aus_holiday.NSW == '1')])
#get_data_api(20180101)


def create_save_docx(contractor_name):
    today_date = datetime.now().strftime('%d-%b-%Y') 
    
    txt = "________________________________________________"
    contractor_name_with_ = (txt[:2] + contractor_name + txt[3 + 1:])[:len(txt)]
    
    find_name    = """<w:t>Contractor</w:t></w:r><w:r w:rsidR="00BC73A7"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:t xml:space="preserve"> Name</w:t></w:r><w:r w:rsidRPr="00CA1BC9"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:t>:</w:t></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/><w:t>________________________________________________</w:t></w:r></w:p><w:p w14:paraId="23E57BA2" w14:textId="77777777" w:rsidR="00CE38E0" w:rsidRDefault="00CE38E0" w:rsidP="00CE38E0"><w:pPr><w:pBdr><w:top w:val="single" w:sz="4" w:space="1" w:color="auto"/><w:left w:val="single" w:sz="4" w:space="0" w:color="auto"/><w:bottom w:val="single" w:sz="4" w:space="1" w:color="auto"/><w:right w:val="single" w:sz="4" w:space="4" w:color="auto"/></w:pBdr><w:ind w:firstLine="720"/><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr></w:pPr></w:p>"""
    replace_name = """<w:t>Contractor</w:t></w:r><w:r w:rsidR="00BC73A7"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:t xml:space="preserve"> Name</w:t></w:r><w:r w:rsidRPr="00CA1BC9"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:t>:</w:t></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/><w:t>{0}</w:t></w:r></w:p><w:p w14:paraId="23E57BA2" w14:textId="77777777" w:rsidR="00CE38E0" w:rsidRDefault="00CE38E0" w:rsidP="00CE38E0"><w:pPr><w:pBdr><w:top w:val="single" w:sz="4" w:space="1" w:color="auto"/><w:left w:val="single" w:sz="4" w:space="0" w:color="auto"/><w:bottom w:val="single" w:sz="4" w:space="1" w:color="auto"/><w:right w:val="single" w:sz="4" w:space="4" w:color="auto"/></w:pBdr><w:ind w:firstLine="720"/><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr></w:pPr></w:p>""".format(contractor_name_with_)
    
    find_client_name    = """<w:t>Client Name:</w:t></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/><w:t xml:space="preserve">                          ________________________________________________ </w:t>"""
    replace_client_name = """<w:t>Client Name:</w:t></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/><w:t xml:space="preserve">                          __NewsCorp______________________________________ </w:t>"""
    
    find_date    = """<w:t>Date</w:t></w:r><w:r w:rsidRPr="00CA1BC9"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:t>:</w:t></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/><w:t>________________________________________________</w:t></w:r></w:p>"""
    replace_date = """<w:t>Date</w:t></w:r><w:r w:rsidRPr="00CA1BC9"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:t>:</w:t></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/></w:r><w:r><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/></w:rPr><w:tab/><w:t>__""" + today_date + """____________________________________</w:t></w:r></w:p>"""
    
    find_approver= """<w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:t>Approved By</w:t></w:r><w:r w:rsidR="0070224C"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:t xml:space="preserve"> Line Manager</w:t></w:r><w:r w:rsidRPr="00653F71"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:t xml:space="preserve"> (Printed Name):</w:t></w:r><w:r w:rsidRPr="00653F71"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:tab/><w:t>___________________________________________</w:t></w:r>"""
    replace_approver= """<w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:t>Approved By</w:t></w:r><w:r w:rsidR="0070224C"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:t xml:space="preserve"> Line Manager</w:t></w:r><w:r w:rsidRPr="00653F71"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:t xml:space="preserve"> (Printed Name):</w:t></w:r><w:r w:rsidRPr="00653F71"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:tab/><w:t>___Anil R Doma_____________________________</w:t></w:r>"""
    
    find_date_signed    = """</w:rPr><w:tab/></w:r><w:r w:rsidRPr="00653F71"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:tab/><w:t>________________________________</w:t></w:r><w:r w:rsidR="00725640">"""
    replace_date_signed = """</w:rPr><w:tab/></w:r><w:r w:rsidRPr="00653F71"><w:rPr><w:rFonts w:ascii="Arial" w:hAnsi="Arial" w:cs="Arial"/><w:sz w:val="18"/><w:szCs w:val="18"/></w:rPr><w:tab/><w:t>___""" + today_date + """__________________</w:t></w:r><w:r w:rsidR="00725640">"""


    #docx_replace("timesheet.docx", "timesheet_new.docx")
    
    zin = zipfile.ZipFile ("timesheet.docx", 'r')
    zout = zipfile.ZipFile ("timesheet_new.docx", 'w')
    
    for item in zin.infolist():
        buffer = zin.read(item.filename)
        if (item.filename == 'word/document.xml'):
            res = buffer.decode("utf-8")
            buffer = res.replace(find_name,replace_name)\
                        .replace(find_client_name,replace_client_name)\
                        .replace(find_date,replace_date)\
                        .replace(find_approver,replace_approver)\
                        .replace(find_date_signed,replace_date_signed)\
                        .encode("utf-8")
        zout.writestr(item, buffer)
    zout.close()
    zin.close()
    
    
    dateList =list(get_week())
    print(dateList)
    #dateList.sort(reverse=True)
    dateList.sort(key=lambda date: datetime.strptime(date, "%d-%b-%Y"),reverse=True)
    
    holiday_flag = [(get_data_api(int(datetime.strptime(d,'%d-%b-%Y').strftime('%Y%m%d')))) for d in dateList]
    
    document = Document("timesheet_new.docx")
    table = document.tables[0]
    j = len(dateList)-1
    for i in range(4,9):
        if holiday_flag[(j)] == 0:
            #print(holiday_flag[i-1])
            table.cell(i,1).text = dateList[j]
            table.cell(i,2).text = "9.00 AM"
            table.cell(i,3).text = "5.30 PM"
            table.cell(i,4).text = "30 Mins"
            table.cell(i,5).text = "8"
            table.cell(i,6).text = "-"
            table.cell(i,7).text = "No"
            j-=1
        else:
            table.cell(i,1).text = ""
            table.cell(i,2).text = ""
            table.cell(i,3).text = ""
            table.cell(i,4).text = ""
            table.cell(i,5).text = ""
            table.cell(i,6).text = ""
            table.cell(i,7).text = ""
            j-=1
    final_output_file_name = 'timesheet_new_{0}_{1}.docx'.format(today_date,contractor_name.replace(" ","_"))
    document.save(final_output_file_name)
    return final_output_file_name
    #! lp final_output_file_name
    #subprocess.check_output(['lp',final_output_file_name])
    


In [4]:
contractor_names =  ["Venkatram Sampathkumar", "Sambaran Roy","Sanjay Kanwar"]
#contractor_names =  ["Sanjay Kanwar"]
for contractor_name in contractor_names:
    print(contractor_name)
    filename = create_save_docx(contractor_name)
    print(filename)

Venkatram Sampathkumar
['22-Oct-2018', '23-Oct-2018', '24-Oct-2018', '25-Oct-2018', '26-Oct-2018']
timesheet_new_22-Oct-2018_Venkatram_Sampathkumar.docx
Sambaran Roy
['22-Oct-2018', '23-Oct-2018', '24-Oct-2018', '25-Oct-2018', '26-Oct-2018']
timesheet_new_22-Oct-2018_Sambaran_Roy.docx
Sanjay Kanwar
['22-Oct-2018', '23-Oct-2018', '24-Oct-2018', '25-Oct-2018', '26-Oct-2018']
timesheet_new_22-Oct-2018_Sanjay_Kanwar.docx


In [20]:
#! lpr timesheet_new_06-Jun-2018_Venkatram_Sampathkumar.docx